In [80]:
import sys

import pandas as pd
from pandas import DataFrame, Series

import numpy as np

import matplotlib.pyplot as plt
import matplotlib

#import xgboost as xgb

from sklearn import tree

%matplotlib inline

In [45]:
train = pd.read_csv("../../data/train.csv", index_col=0)
test = pd.read_csv("../../data/test.csv", index_col=0)
train.head(2)

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby


In [46]:
test.head(2)

,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
ID,,,,,,,
1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan


In [51]:
train.SexuponOutcome.value_counts()

Neutered Male    9779
Spayed Female    8820
Intact Male      3525
Intact Female    3511
Unknown          1093
Name: SexuponOutcome, dtype: int64

In [30]:
#data[data.SexuponOutcome == "Unknown"].AnimalType.value_counts()

In [56]:
opt = train.copy()
opt.SexuponOutcome = opt.SexuponOutcome.replace("Neutered Male", 1)
opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Male", 1)
opt.SexuponOutcome = opt.SexuponOutcome.replace("Spayed Female", 2)
opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Female", 2)
opt.SexuponOutcome = opt.SexuponOutcome.replace("Unknown", 3)

opt.head()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,1.0,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,2.0,1 year,Domestic Shorthair Mix,Cream Tabby
A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,1.0,2 years,Pit Bull Mix,Blue/White
A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,1.0,3 weeks,Domestic Shorthair Mix,Blue Cream
A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,1.0,2 years,Lhasa Apso/Miniature Poodle,Tan


In [87]:
opt.OutcomeType = opt.OutcomeType.astype("category")


In [94]:
opt.OutcomeTypeIndex = opt.OutcomeType.cat.codes.astype(int)
opt

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,AnimalTypeIndex
AnimalID,,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,1.0,1 year,Shetland Sheepdog Mix,Brown/White,Dog
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,2.0,1 year,Domestic Shorthair Mix,Cream Tabby,Cat
A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,1.0,2 years,Pit Bull Mix,Blue/White,Dog
A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,1.0,3 weeks,Domestic Shorthair Mix,Blue Cream,Cat
A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,1.0,2 years,Lhasa Apso/Miniature Poodle,Tan,Dog
A677334,Elsa,2014-04-25 13:04:00,Transfer,Partner,Dog,2.0,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan,Dog
A699218,Jimmy,2015-03-28 13:11:00,Transfer,Partner,Cat,1.0,3 weeks,Domestic Shorthair Mix,Blue Tabby,Cat
A701489,NaN,2015-04-30 17:02:00,Transfer,Partner,Cat,3.0,3 weeks,Domestic Shorthair Mix,Brown Tabby,Cat
A671784,Lucy,2014-02-04 17:17:00,Adoption,NaN,Dog,2.0,5 months,American Pit Bull Terrier Mix,Red/White,Dog
